In [2]:
import os
import json
import pandas as pd
import altair as alt
import pulp
import numpy as np

In [3]:
def import_data(file_name: str):
    try:
        with open(file_name, 'r') as file:
            data = json.load(file)
        return pd.DataFrame(data)
    
    except FileNotFoundError:
        print(f"Error: The file '{file_name}' was not found.")

In [4]:
allocation_plan = import_data('allocation_plan.json')
asset_rates = import_data('asset_rates.json')
asset_uptime = import_data('asset_uptime.json')
orders = import_data('orders.json')
skus = import_data('skus.json')

In [5]:
# Calulating asset working hours per year
asset_uptime['hours_per_year'] = (asset_uptime['days_per_week'] * asset_uptime['weeks_per_year'] * asset_uptime['hours_per_shift'] * asset_uptime['shifts_per_day'])
asset_uptime['hours_per_month'] = (asset_uptime['hours_per_year']/12)

In [6]:
asset_rates_merged = asset_rates.merge(skus, on='Product', how='left')
asset_rates_merged['hours_per_batch'] = ((asset_rates_merged['Lot Size'] / asset_rates_merged['run_rate']) + asset_rates_merged['cleanup_time'])
asset_rates_merged.head()

,asset_id,Product,run_rate,cleanup_time,Lot Size,hours_per_batch
0,L007,A,45,1.0,4000,89.888889
1,L007,B,45,1.0,6000,134.333333
2,L007,C,45,1.5,1000,23.722222
3,L007,D,45,1.5,6500,145.944444
4,L007,E,45,3.0,15000,336.333333


In [7]:
demand_by_product = orders.groupby(['Year', 'Month', 'SKU'])['Demand'].sum().drop(columns = ['proj_id', 'Date']).reset_index()
demand_by_product.head(6)

,Year,Month,SKU,Demand
0,2025,1,A,1701
1,2025,1,B,3483
2,2025,1,C,8863
3,2025,1,D,7414
4,2025,1,E,10393
5,2025,2,A,7537


In [8]:
demand_by_product = orders.groupby(['Year', 'Month', 'SKU'])['Demand'].sum().drop(columns = ['proj_id', 'Date']).reset_index()
demand_by_product.head(6)

demand_map = demand_by_product.set_index(['SKU', 'Year', 'Month'])['Demand'].to_dict()
capacity_map = asset_uptime.set_index('asset_id')['hours_per_month'].to_dict()


# Create Sets (Indexes)
products = skus['Product'].unique()
assets = asset_uptime['asset_id'].unique()
years = demand_by_product['Year'].unique() 
months = demand_by_product['Month'].unique() 

# Create mapping dictionaries for referencing lot size and hours per batch
lot_size_map = skus.set_index('Product')['Lot Size'].to_dict()
hours_per_batch_map = asset_rates_merged.set_index(['asset_id', 'Product'])['hours_per_batch'].to_dict()

# Setup Optimization Problem - minimzation (of total number of hours)
model = pulp.LpProblem("Asset_Allocation", pulp.LpMinimize)

# Define Decision Variables
X = pulp.LpVariable.dicts("Units", (products, assets, years, months), lowBound=0, cat='Continuous')  # keep as continuous so that it is easier to solve (not integer-integer)
B = pulp.LpVariable.dicts("Batches", (products, assets, years, months), lowBound=0, cat='Integer')

# Objective Function: Minimize total production time
model += (
    pulp.lpSum([
        B[i][j][y][m] * hours_per_batch_map.get((j, i), 0)
        for i in products for j in assets for y in years for m in months
    ]), "Total_Production_Hours"
)

# Constraints:
# Demand Constraint: Must meet all demand in a given year (X >= total demand)
for i in products:
    for y in years:
        for m in months:
            demand = demand_map.get((i, y, m), 0)
            model += (
                pulp.lpSum([X[i][j][y][m] for j in assets]) == demand,
                f"Demand_Met_{i}_{y}_{m}"
            )

# Capacity Constraint: Cannot exceed annual capacity on any asset within the year
for j in assets:
    capacity = capacity_map.get(j, 0)
    for y in years:
        for m in months:
            model += (
                pulp.lpSum([
                    B[i][j][y][m] * hours_per_batch_map.get((j, i), 0)
                    for i in products ]) <= capacity,
                f"Capacity_Limit_{j}_{y}_{m}"
            )

# Batch constraint: Must have sufficient batches for number of units (round up) Batches >= X/(lot size)
for i in products:
    lot_size = lot_size_map.get(i, np.inf) 
    if lot_size <= 0 or lot_size == np.inf:
        continue
        
    for j in assets:
        for y in years:
            for m in months:
                model += (
                    lot_size * B[i][j][y][m] >= X[i][j][y][m],
                    f"Batch_Link_{i}_{j}_{y}_{m}"
                )


# Solver
model.solve()   
total_hours = pulp.value(model.objective)
print(f'Total Hours (minimized): {total_hours}')
# Extract and format results into a DataFrame
results = []
for i in products:
    for j in assets:
        for y in years:
            for m in months:
                batches = B[i][j][y][m].varValue
                if batches > 0:
                    hours_required = batches * hours_per_batch_map.get((j, i))
                    units = X[i][j][y][m].varValue
                    results.append({'Asset': j, 'Product': i, 'Year': y, 'Month': m, 'Units_Allocated': units,'Batches_Run': batches,'Hours_Required': hours_required })
                
df_results = pd.DataFrame(results)
df_results.head(6)

Total Hours (minimized): 55758.98120064706


,Asset,Product,Year,Month,Units_Allocated,Batches_Run,Hours_Required
0,L042,A,2025,1,1701.000,0.425250,23.530500
1,L042,A,2025,2,7537.000,1.884250,104.261833
2,L042,A,2025,3,24841.550,6.210387,343.641436
3,L042,A,2025,4,8578.000,2.144500,118.662333
4,L042,A,2025,5,18228.080,4.557020,252.155101
5,L042,A,2025,6,18987.398,4.746849,262.659000


In [9]:
df_results['Asset'].unique()

array(['L042', 'L673', 'L451', 'L007'], dtype=object)

Visualize Annually

In [10]:
df_results.head()

,Asset,Product,Year,Month,Units_Allocated,Batches_Run,Hours_Required
0,L042,A,2025,1,1701.00,0.425250,23.530500
1,L042,A,2025,2,7537.00,1.884250,104.261833
2,L042,A,2025,3,24841.55,6.210387,343.641436
3,L042,A,2025,4,8578.00,2.144500,118.662333
4,L042,A,2025,5,18228.08,4.557020,252.155101


In [11]:
annual_utiization = df_results.groupby(['Product', 'Asset', 'Year'])['Hours_Required'].sum().reset_index()

In [12]:
annual_uptime = asset_uptime[['asset_id', 'hours_per_year']]
annual_utiization = annual_utiization.merge(annual_uptime, left_on='Asset', right_on='asset_id').rename(columns = {'hours_per_year' : 'annual uptime'}).drop(columns='asset_id')
annual_utiization.head()

,Product,Asset,Year,Hours_Required,annual uptime
0,A,L042,2025,1841.534269,4680.0
1,A,L042,2026,1489.949588,4680.0
2,A,L042,2027,904.412206,4680.0
3,A,L042,2028,1178.741703,4680.0
4,A,L042,2029,1626.727436,4680.0


In [13]:
annual_utiization['utilization_ratio'] = annual_utiization['Hours_Required']/ annual_utiization['annual uptime']

In [14]:
Utilization_chart =  alt.Chart(annual_utiization).mark_line().encode(
        alt.X('Year:O', title='Year' ),
        y = alt.Y('utilization_ratio:Q', title = 'Utilization Ratio'),
        color = 'Asset').properties(width = 500, title = 'Suggested Asset Capacity Utilization by Asset and Year')

over_utilized = pd.DataFrame({'y': [1.0]})
horizontal_line = alt.Chart(over_utilized).mark_rule(color='black', strokeDash=[5, 5]).encode(
    y = alt.Y('y:Q'))

# Utilization_chart + horizontal_line
Utilization_chart

alt.Chart(...)

In [15]:
Utilization_chart.save('Suggested Utilization Chart.png')